### Example of PySpark ML

In [1]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Pactice6').getOrCreate()

In [2]:
# Read the dataset
train = spark.read.csv('data1.csv', header=True,inferSchema=True)

In [3]:
train.show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
|Trevor| 31|        10| 30000|
|  Adam| 30|         8| 25000|
|   Eve| 29|         4| 20000|
|   Max| 24|         3| 20000|
|   Joe| 21|         1| 15000|
|   Bob| 23|         2| 18000|
+------+---+----------+------+



In [4]:
train.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"], outputCol="Independent Features")

In [6]:
output = featureassembler.transform(train)

In [7]:
output.show()

+------+---+----------+------+--------------------+
|  Name|Age|Experience|Salary|Independent Features|
+------+---+----------+------+--------------------+
|Trevor| 31|        10| 30000|         [31.0,10.0]|
|  Adam| 30|         8| 25000|          [30.0,8.0]|
|   Eve| 29|         4| 20000|          [29.0,4.0]|
|   Max| 24|         3| 20000|          [24.0,3.0]|
|   Joe| 21|         1| 15000|          [21.0,1.0]|
|   Bob| 23|         2| 18000|          [23.0,2.0]|
+------+---+----------+------+--------------------+



In [9]:
final_data = output.select("Independent Features","Salary")

In [10]:
final_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [20]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data,test_data = final_data.randomSplit([0.75,0.25])
regressor= LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [21]:
### coefficients
regressor.coefficients

DenseVector([-64.8464, 1584.7554])

In [22]:
### Intercepts
regressor.intercept

15414.10693970376

In [23]:
test_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [24.0,3.0]| 20000|
+--------------------+------+



In [25]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [26]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [24.0,3.0]| 20000|18612.05915813422|
+--------------------+------+-----------------+



In [27]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1387.9408418657804, 1926379.7805190913)